# Topic Modeling with LDA
Use this notebook to practice topic modeling (see lecture slides for reference). First, the usual stuff.

In [1]:
import pandas as pd
transcript_df = pd.read_excel('./data/sample_transcript.xlsx')
transcript_df.sample(3)

,timestamp,speaker,utterance
236,00:12:03,Charlie,Like the round...
62,00:03:34,Alpha,We can do that.
507,00:34:26,Bravo,"Yeah, it's it's 6 now. It's 6 here."


In [2]:
utterances = transcript_df['utterance'].tolist()
utterances[:3]

['All right.', 'Start recording.', "That's alright."]

In [3]:
def remove_puncts(utterance_text, alphanumeric_only='True'):
    utterance_text = utterance_text.replace('-', ' ')
    clean_utterance_text = ''.join(e for e in utterance_text if e.isalnum() or e == ' ').lower()
    clean_utterance_text = ' '.join(clean_utterance_text.split())
    return clean_utterance_text

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def get_words_tokenized_nopunct_nostop(utterances, stop_w=stop_words):
    utterance_words_list = []
    for utterance in utterances:
        clean_utterance = remove_puncts(utterance)
        words = word_tokenize(clean_utterance.lower())
        words_nostop = [word for word in words if not word in stop_w]
        utterance_words_list.append(words_nostop)
    return utterance_words_list

In [5]:
tokenized_utterances_list = get_words_tokenized_nopunct_nostop(utterances)

## Train an LDA Model
LDA models require an estimated number of topics. We can choose this to be 5, and fine-tune if we see overlaps.

In [6]:
import gensim

dictionary = gensim.corpora.Dictionary(tokenized_utterances_list)
corpus = [dictionary.doc2bow(utterance) for utterance in tokenized_utterances_list]
topic_count = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topic_count, id2word=dictionary, passes=50)

## Visualize the LDA Model
We use the famouse [pyLDAvis](https://pypi.org/project/pyLDAvis/) library to make sense of the topics.

First, we set up the notebook to suppress warnings (you'll get a lot due to package deprecations).

In [7]:
import warnings
warnings.filterwarnings('ignore')

## IMPORTANT: Set the slider below to a $\lambda$ value of 0.4
A $\lambda$ of 1 shows all frequently-occurring words, which gives you a global picture of the text in general, but does not help you understand the topics. A $\lambda$ of 0.1 shows only the words unique to each topic, which helps you understand the difference between the topics, but not what the topics are about. A $\lambda$ of 0.4 achieves a good balance. See [this paper](https://aclanthology.org/W14-3110.pdf) for more details.

In [8]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
gensimvis.prepare(ldamodel, corpus, dictionary)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/externals/loky/backend/fork_exec.py:38: DeprecationWarning: This process (pid=33509) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/joblib/_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <numbe

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.082993 -0.207054       1        1  27.015658
1     -0.236061 -0.023812       2        1  21.638319
3      0.090393  0.070588       3        1  19.017687
4      0.033581  0.115754       4        1  18.433935
0      0.029093  0.044523       5        1  13.894401, topic_info=         Term        Freq       Total Category  logprob  loglift
34       yeah  106.000000  106.000000  Default  30.0000  30.0000
32       like   85.000000   85.000000  Default  29.0000  29.0000
13         na   17.000000   17.000000  Default  28.0000  28.0000
11        gon   17.000000   17.000000  Default  27.0000  27.0000
70        one   28.000000   28.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
77       work    2.502389    8.035372   Topic5  -4.8848   0.8071
38      maybe    2.280335   21.987622   Topic5  -4.9777  -0.2925
39        put    1.838125   15.753228   Topic5  -5.1933  -0.1746
105       two    1.766889    6.448487   Topic5  -5.2328   0.6791
43   laughter    1.766742   11.237392   Topic5  -5.2329   0.1236

[270 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
587       5  0.624939      10
457       4  0.826948       3
85        1  0.938296      3d
154       4  0.743139       4
472       2  0.846956       6
...     ...       ...     ...
34        2  0.988749    yeah
116       1  0.750730  yellow
14        1  0.869336     yes
14        4  0.108667     yes
500       1  0.833960     yet

[336 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 5, 1])

## View topic distribution for a document (review)

In [9]:
ldamodel.get_document_topics(corpus[0])

[(0, 0.10102035),
 (1, 0.10000096),
 (2, 0.59730875),
 (3, 0.10131063),
 (4, 0.10035934)]

# Exercise: What are some topics relevant to you?
Explore the topics and their keywords. 